# 코스콤 틱데이터 처리 - 빠른 시작 가이드

이 노트북은 코스콤 틱데이터를 HFT 전략에 활용하기 위한 기본 사용법을 안내합니다.

## 📋 필수 패키지 설치

```bash
pip install pandas pyarrow polars duckdb numba tqdm
```

In [ ]:
# 라이브러리 임포트
from hft_tick_processor_v2 import TickDataConverter, MultiFileTickLoader
import polars as pl
import sys

---

## 1단계: 데이터 전처리 (GZ → Parquet)

⚠️ **주의**: 이 단계는 시간이 오래 걸립니다 (분기당 10~30분). 한 번만 실행하면 됩니다.

In [ ]:
# ============================================================================
# 전처리 설정
# ============================================================================

TickDataConverter.batch_convert(
    # ========== 필수 인자 ==========
    input_dir=r"E:\선물 체결틱데이터(2010.Q1~2023.Q4)",
    # 원본 .dat.gz 파일들이 있는 디렉토리 경로
    # 예: r"E:\선물 체결틱데이터(2010.Q1~2023.Q4)"
    #     r"E:\유가증권 체결틱데이터(2010.1~2023.12)"
    
    output_dir=r"E:\parquet\futures",
    # Parquet 파일을 저장할 디렉토리 (없으면 자동 생성)
    # 권장: 원본과 별도의 폴더에 저장
    
    is_derivative=True,
    # True: 선물/옵션 데이터 (21~25개 필드)
    # False: 유가증권/코스닥 데이터 (45개 필드)
    
    # ========== 선택 인자 (기간 필터) ==========
    start_year=2017,
    # 시작 연도 (포함)
    # None이면 모든 연도 포함
    # 예: 2017 → 2017년부터 시작
    
    end_year=2019,
    # 종료 연도 (포함)
    # None이면 모든 연도 포함
    # 예: 2019 → 2019년까지
    
    quarters=[1, 2],
    # 분기 리스트 (선물/옵션 데이터용)
    # None이면 모든 분기 포함
    # 예: [1] → 1분기만
    #     [1, 2] → 1,2분기만
    #     [1, 2, 3, 4] → 전체
    # 주의: quarters와 months는 동시에 사용 불가
    
    # months=[1, 2, 3],
    # 월 리스트 (주식 데이터용)
    # None이면 모든 월 포함
    # 예: [1, 2, 3] → 1~3월만
    #     [6, 7, 8] → 6~8월만
    # 주의: quarters와 months는 동시에 사용 불가
    
    # ========== 고급 인자 ==========
    # pattern="*.dat.gz",
    # 파일명 패턴 (기본값으로 충분)
    # 예: "*.dat.gz", "*_Q1.dat.gz"
)

### 💡 전처리 팁

1. **처음 사용**: 테스트로 1년치만 먼저 변환 (`start_year=2017, end_year=2017`)
2. **디스크 공간**: 원본의 약 20% 필요 (500GB → 100GB)
3. **시간 예상**: 분기당 10~30분, 전체 변환은 하룻밤 소요
4. **중단 가능**: 이미 변환된 파일은 자동으로 스킵됨

---

## 2단계: 데이터 쿼리 및 분석

### 💾 메모리 체크 함수

In [ ]:
def get_dataframe_memory_mb(df):
    """DataFrame의 메모리 사용량 계산 (MB)"""
    if hasattr(df, 'estimated_size'):  # Polars
        return df.estimated_size() / (1024 ** 2)
    else:  # Pandas
        return df.memory_usage(deep=True).sum() / (1024 ** 2)

def print_memory_status(df, label="DataFrame"):
    """메모리 상태 출력"""
    memory_mb = get_dataframe_memory_mb(df)
    memory_gb = memory_mb / 1024
    
    print(f"\n{'='*80}")
    print(f"📊 {label} 메모리 사용량")
    print(f"{'='*80}")
    print(f"행 수: {len(df):,}개")
    print(f"컬럼 수: {len(df.columns)}개")
    print(f"메모리: {memory_mb:,.1f} MB ({memory_gb:.2f} GB)")
    
    # 메모리 경고
    if memory_gb > 10:
        print(f"⚠️  경고: 메모리 사용량이 큽니다! ({memory_gb:.2f} GB)")
        print(f"   → 필터 조건을 더 좁히거나 컬럼 수를 줄이세요.")
    elif memory_gb > 5:
        print(f"⚠️  주의: 메모리 사용량이 높습니다. ({memory_gb:.2f} GB)")
    else:
        print(f"✅ 메모리 사용량 정상")
    
    print(f"{'='*80}\n")

### 📈 데이터 로딩 (Polars)

In [ ]:
# ============================================================================
# Polars로 데이터 로딩
# ============================================================================

df = MultiFileTickLoader.load_period_polars(
    # ========== 필수 인자 ==========
    parquet_dir=r"E:\parquet\futures",
    # Parquet 파일들이 있는 디렉토리
    # 1단계의 output_dir과 동일한 경로
    
    start_year=2017,
    # 로딩할 시작 연도 (포함)
    # 예: 2017 → 2017년부터
    
    end_year=2019,
    # 로딩할 종료 연도 (포함)
    # 예: 2019 → 2019년까지
    # 주의: 전처리하지 않은 연도는 로드 불가
    
    # ========== 선택 인자 (필터링) ==========
    filters={
        'ISIN_CODE': 'KR4101M30004'
    },
    # 조건 필터 (딕셔너리 형태)
    # None이면 전체 데이터 로드 (메모리 주의!)
    # 
    # 사용 가능한 필터 필드:
    #   - ISIN_CODE: 종목코드
    #       예: 'KR4101M30004' (단일 종목)
    #           ['KR4101M30004', 'KR4101M60009'] (여러 종목)
    #   - TRADE_DATE: 거래일자
    #       예: '20170315' (특정일)
    #           ['20170315', '20170316'] (여러 날)
    #   - LST_ASKBID_TP_CD: 매도/매수 구분
    #       예: '1' (매수 체결만)
    #           '2' (매도 체결만)
    #
    # 여러 조건 동시 사용:
    #   filters={
    #       'ISIN_CODE': 'KR4101M30004',
    #       'LST_ASKBID_TP_CD': '1'  # 해당 종목의 매수 체결만
    #   }
    
    columns=['TRADE_DATE', 'TRD_TM', 'TRD_PRC', 'TRDVOL', 'LST_ASKBID_TP_CD']
    # 읽을 컬럼 리스트
    # None이면 모든 컬럼 로드 (메모리 사용량 증가)
    # 
    # ========== 파생상품 사용 가능한 컬럼 ==========
    # 기본 정보:
    #   - TRADE_DATE: 거래일자 (YYYYMMDD)
    #   - ISIN_CODE: 종목코드 (예: KR4101M30004)
    #   - JONG_INDEX: 종목인덱스
    #
    # 체결 정보:
    #   - TRD_TM: 체결시각 (HHMMSSmmm, 예: 090012345 = 09:00:12.345)
    #   - TRD_PRC: 체결가격
    #   - TRDVOL: 체결수량 (계약수)
    #   - LST_ASKBID_TP_CD: 최종매도매수구분 ('1'=매수, '2'=매도)
    #
    # 당일 OHLC:
    #   - OPEN_PRICE: 시가
    #   - HIGH_PRICE: 고가
    #   - LOW_PRICE: 저가
    #
    # 누적 정보:
    #   - ACC_TRDVOL: 누적체결수량 (장 시작부터)
    #   - ACC_AMT: 누적거래대금 (원)
    #
    # ========== 주식 추가 컬럼 (is_derivative=False인 경우) ==========
    #   - BID_MBR_NO: 매수회원번호
    #   - ASK_MBR_NO: 매도회원번호
    #   - BIDORD_TP_CD: 매수호가유형코드
    #   - ASKORD_TP_CD: 매도호가유형코드
    #   - BID_INVST_TP_CD: 매수투자자구분코드
    #   - ASK_INVST_TP_CD: 매도투자자구분코드
    #   등 (총 45개 필드 - koscom_tick_parser.py 참조)
    #
    # 💡 팁: 필요한 컬럼만 선택하면 메모리 절약!
    #   예: ['TRD_TM', 'TRD_PRC', 'TRDVOL'] (최소 3개)
)

# 메모리 사용량 출력
print_memory_status(df, "로드된 틱데이터")

### 📊 데이터 미리보기

In [ ]:
# 처음 10개 행
print("\n처음 10개 행:")
print(df.head(10))

# 기본 통계
print("\n기본 통계:")
print(df.describe())

---

## 3단계: 간단한 분석 예시

### 📈 시간대별 거래량 분석

In [ ]:
# 시간(HH) 추출
df_analysis = df.with_columns([
    pl.col('TRD_TM').str.slice(0, 2).cast(pl.Int32).alias('hour')
])

# 시간대별 집계
hourly_stats = df_analysis.group_by('hour').agg([
    pl.count().alias('trade_count'),
    pl.col('TRDVOL').sum().alias('total_volume'),
    pl.col('TRD_PRC').mean().alias('avg_price'),
    pl.col('TRD_PRC').std().alias('price_volatility')
]).sort('hour')

print("\n시간대별 통계:")
print(hourly_stats)

### 💹 매수/매도 압력 분석

In [ ]:
# 매수/매도 구분 집계
buysell_stats = df.group_by('LST_ASKBID_TP_CD').agg([
    pl.count().alias('count'),
    pl.col('TRDVOL').sum().alias('total_volume'),
    pl.col('TRD_PRC').mean().alias('avg_price')
])

print("\n매수/매도 구분 통계:")
print("  1 = 매수 체결")
print("  2 = 매도 체결")
print(buysell_stats)

### 💾 Pandas로 변환 (필요시)

In [ ]:
# Polars → Pandas 변환
df_pandas = df.to_pandas()

print("\nPandas DataFrame으로 변환 완료")
print_memory_status(df_pandas, "Pandas DataFrame")

# Pandas 작업 예시
print("\nPandas 기본 정보:")
print(df_pandas.info())

---

## 💡 추가 예시

### 여러 종목 동시 로드

In [ ]:
# 여러 종목 로드
df_multi = MultiFileTickLoader.load_period_polars(
    parquet_dir=r"E:\parquet\futures",
    start_year=2017,
    end_year=2017,
    filters={
        'ISIN_CODE': ['KR4101M30004', 'KR4101M60009']  # 리스트로 여러 종목
    },
    columns=['ISIN_CODE', 'TRD_TM', 'TRD_PRC', 'TRDVOL']
)

print_memory_status(df_multi, "여러 종목 데이터")

# 종목별 통계
symbol_stats = df_multi.group_by('ISIN_CODE').agg([
    pl.count().alias('trade_count'),
    pl.col('TRDVOL').sum().alias('total_volume')
])

print("\n종목별 통계:")
print(symbol_stats)

### 특정 시간대만 필터링

In [ ]:
# 데이터 로드 후 추가 필터링 (09:00 ~ 10:00)
df_morning = df.filter(
    (pl.col('TRD_TM') >= '090000000') &
    (pl.col('TRD_TM') < '100000000')
)

print(f"\n9시대 데이터: {len(df_morning):,}개 행")
print_memory_status(df_morning, "9시대 데이터")

---

## 🎯 다음 단계

1. **백테스팅**: `hft_strategy_guide.py` 참조
2. **고급 전략**: VWAP, Order Flow Imbalance 등
3. **성능 최적화**: Numba로 계산 가속화